In [160]:
#import all packages here 
import pandas as pd 
import numpy as np 
import json
import requests
from pandas.io.json import json_normalize
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 
import numpy as np 
import matplotlib.pyplot as plt  

In [1]:
# !conda install -c conda-forge folium=0.5.0 --yes 
! pip install folium

### Introduction
A new coronavirus designated 2019-nCoV was first identified in Wuhan, the capital of China's Hubei province. The virus has shown evidence of human-to-human transmission, and its transmission rate (rate of infection) appeared to escalate in mid-January 2020.

**Question:** Can different cities within the United States with more than 275 Coronavirus cases be clustered into groups based on the types of venues in each city? Is there any relationship between the average number of cases in a city and the cluster the city is assigned to? 

### Data

<br>-Kaggle time series dataset that provides data about the number of confirmed Coronavirus cases to date. The dataset can be found here: https://www.kaggle.com/imdevskp/corona-virus-report
<br>**Province_State:** Name of the Province/State (string)
<br>**Country_Region:** Name of Country/Region *we will only use US* (string)
<br>**Lat:** Latitude (Float)
<br>**Long:** Longitude (Float)
<br>**Combined Key:** Name of City,State,Country (string)
<br>**9/23/20:** Number of cases confirmed as of September 23, 2020 (Int)

<br>-Forsquare Places API
<br>Group: venues, Endpoints: explore.
<br>**Description:** Returns a list of recommended venues near the current location, within a certain radius.
<br>More information on this API, including request, authentication, and parameters can be found here: https://developer.foursquare.com/docs/api/venues/explore

### Methodology 

**Load  and Clean Data**

- Download csv from Kaggle 
- Load csv into python as a dataframe 
- Drop the columns that are not needed as part of the anlysis. 
- Filter by: Cities that have > 275 coronavirus cases 
- Drop rows that do not have latitude/longitude data 

In [161]:
df = pd.read_csv('time_series_covid_19_confirmed_US_updated.csv')

df = df[['Province_State','Country_Region','Lat','Long_','Combined_Key','9/23/20']] #get relevant columns
df.columns = ['State','Country','Lat', 'Long','Area','Cases'] #update column names 
df = df[df['Cases']>275] # filter by >275 cases on 9/23  
df = df[df['Lat'] != 0.0] # drop values with no latitude data
df = df[df['Long'] != 0.0] # drop values with no longitude data

df = df.reset_index(drop=True) #drop index

#split data frame in half due to Forsquare API limitations
df_firsth = df[0:939]
df_secondh = df[940:1879]

df.head()

,State,Country,Lat,Long,Area,Cases
0,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",1715
1,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",5141
2,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",851
3,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",638
4,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",1580


**Forsquare API Setup**

Import Forsquare Information and credentials in order to use the forsquare api (described above) 

In [162]:
#split the dataset up in two halves with different client ID/SECRET due to the amount of data being used 

CLIENT_ID_firsth = 'F4U2RBBBO3AAAOUCQK2TQVFHEKVNHNQ0HGZQWNHSWEYIXEQR'
CLIENT_SECRET_firsth = '1L32O1ILJTDCTZRKQ1DMFSCWY4AQ2GDBUY0544LSXSLVJS4A'

CLIENT_ID_secondh = 'LC4FON34WZMTTBJ4RXABTC3Z4JJT5P0TUILSRNVJCZJ4RGKS'
CLIENT_SECRET_secondh = '35YOULZKKGQ3X2RPREIZAMERXLX0RKRBTAECQDGW0GGRL0I5'

VERSION = '20180605'

**Extract Venues**

Use the Forsquare API to extract the nearby venues for each city. Create a function to get nearby venues within a radius of 5000 to the location of the coronavirus outbreak. Use the function to get all the nearby venues within that radius. After the venues have been extracted, transform the data to group by each city. 

In [163]:
def getNearbyVenues(names, latitudes, longitudes, CLIENT_ID, CLIENT_SECRET, VERSION = '20180605', radius=5000, limit=100):
    num = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        num +=1
        print(num)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Province/State', 
                  'Province/State Latitude', 
                  'Province/State Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
#get venues for firsth
venues_firsth = getNearbyVenues(names=df_firsth['Area'],latitudes=df_firsth['Lat'],longitudes=df_firsth['Long'], CLIENT_ID = CLIENT_ID_firsth, CLIENT_SECRET = CLIENT_SECRET_firsth)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [11]:
#get venues for second half 
venues_secondh = getNearbyVenues(names=df_secondh['Area'],latitudes=df_secondh['Lat'],longitudes=df_secondh['Long'], CLIENT_ID = CLIENT_ID_secondh, CLIENT_SECRET = CLIENT_SECRET_secondh)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [164]:
venues = pd.concat([venues_firsth,venues_secondh]) #combine both data frames
venues.groupby('Province/State').count() #group by each city,state to get the number of venues 

,Province/State Latitude,Province/State Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Province/State,,,,,,
"Abbeville, South Carolina, US",4,4,4,4,4,4
"Accomack, Virginia, US",9,9,9,9,9,9
"Ada, Idaho, US",1,1,1,1,1,1
"Adair, Kentucky, US",18,18,18,18,18,18
"Adair, Missouri, US",48,48,48,48,48,48
...,...,...,...,...,...,...
"York, South Carolina, US",30,30,30,30,30,30
"York, Virginia, US",44,44,44,44,44,44
"Young, Texas, US",1,1,1,1,1,1


In [165]:
print('There are {} uniques categories of venues.'.format(len(venues['Venue Category'].unique())))

There are 532 uniques categories of venues.


**One hot encoding**

One hot encoding is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction. Use the extracted venues to conduct one hot encoding. 

In [166]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="") # one hot encoding
onehot['Province/State'] = venues['Province/State'] # add Province/State column back to dataframe
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1]) # move Province/State column to the first column
onehot = onehot[fixed_columns]
onehot.head()

,Province/State,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Autauga, Alabama, US",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Baldwin, Alabama, US",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Baldwin, Alabama, US",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Baldwin, Alabama, US",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Barbour, Alabama, US",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [167]:
grouped = onehot.groupby('Province/State').mean().reset_index() #group onehot encodings by Province/State
grouped.head()

,Province/State,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,"Abbeville, South Carolina, US",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,"Accomack, Virginia, US",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Ada, Idaho, US",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"Adair, Kentucky, US",0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,"Adair, Missouri, US",0.020833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.0


**Top Venues**

View the top 30 venues for each location from all the nearby venues

In [168]:
num_top_venues_toView = 30

for hood in grouped['Province/State']:
    print("----"+hood+"----")
    temp = grouped[grouped['Province/State'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues_toView))
    print('\n')

----Abbeville, South Carolina, US----
                            venue  freq
0                            Farm  0.75
1                         Stables  0.25
2                             ATM  0.00
3                    Outlet Store  0.00
4               Paella Restaurant  0.00
5                 Paintball Field  0.00
6            Pakistani Restaurant  0.00
7                          Palace  0.00
8   Paper / Office Supplies Store  0.00
9           Outdoors & Recreation  0.00
10                           Park  0.00
11                        Parking  0.00
12                    Pastry Shop  0.00
13                      Pawn Shop  0.00
14               Pedestrian Plaza  0.00
15                  Pelmeni House  0.00
16                    Outlet Mall  0.00
17           Outdoor Supply Store  0.00
18        New American Restaurant  0.00
19              Outdoor Sculpture  0.00
20              Other Repair Shop  0.00
21                Other Nightlife  0.00
22           Other Great Outdoors  0.00
23

                                       venue  freq
0                              Deli / Bodega  0.06
1                                College Gym  0.06
2                                        Gym  0.06
3                                Golf Course  0.06
4                        American Restaurant  0.06
5                 Construction & Landscaping  0.03
6                                Planetarium  0.03
7                                   Gym Pool  0.03
8                              Design Studio  0.03
9                              Movie Theater  0.03
10                              Gourmet Shop  0.03
11  Residential Building (Apartment / Condo)  0.03
12                   New American Restaurant  0.03
13                               Gas Station  0.03
14                             Garden Center  0.03
15                                      Café  0.03
16                       Sporting Goods Shop  0.03
17                               Flower Shop  0.03
18                             

                            venue  freq
0                  Discount Store   0.4
1                    Burger Joint   0.2
2                          Office   0.2
3                   Grocery Store   0.2
4                             ATM   0.0
5   Paper / Office Supplies Store   0.0
6               Paella Restaurant   0.0
7                 Paintball Field   0.0
8            Pakistani Restaurant   0.0
9                          Palace   0.0
10                        Parking   0.0
11                           Park   0.0
12                    Pastry Shop   0.0
13                      Pawn Shop   0.0
14               Pedestrian Plaza   0.0
15                  Pelmeni House   0.0
16          Performing Arts Venue   0.0
17                   Outlet Store   0.0
18                    Outlet Mall   0.0
19             Persian Restaurant   0.0
20          Outdoors & Recreation   0.0
21           Outdoor Supply Store   0.0
22              Outdoor Sculpture   0.0
23              Other Repair Shop   0.0


                            venue  freq
0            Fast Food Restaurant  0.31
1                  Discount Store  0.15
2                        Pharmacy  0.08
3                     Pizza Place  0.08
4                  Ice Cream Shop  0.08
5             Fried Chicken Joint  0.08
6                     Supermarket  0.08
7                           Trail  0.08
8                  Sandwich Place  0.08
9   Paper / Office Supplies Store  0.00
10                         Palace  0.00
11                           Park  0.00
12                        Parking  0.00
13           Outdoor Supply Store  0.00
14           Pakistani Restaurant  0.00
15                Paintball Field  0.00
16              Paella Restaurant  0.00
17                   Outlet Store  0.00
18                    Outlet Mall  0.00
19          Outdoors & Recreation  0.00
20              Outdoor Sculpture  0.00
21                      Pawn Shop  0.00
22              Other Repair Shop  0.00
23                Other Nightlife  0.00


                            venue  freq
0                             Bar  0.15
1                          Bakery  0.05
2                            Food  0.05
3                  Baseball Field  0.05
4                            Bank  0.05
5                 Harbor / Marina  0.05
6                     Snack Place  0.05
7            Fast Food Restaurant  0.05
8                   Grocery Store  0.05
9                     Pizza Place  0.05
10                Motorcycle Shop  0.05
11                   Liquor Store  0.05
12                 Sandwich Place  0.05
13                    Post Office  0.05
14                    Golf Course  0.05
15                   Intersection  0.05
16            American Restaurant  0.05
17                          Diner  0.05
18              Outdoor Sculpture  0.00
19  Paper / Office Supplies Store  0.00
20                         Notary  0.00
21                         Office  0.00
22                      Pawn Shop  0.00
23                    Opera House  0.00


...

29                         Notary  0.00


----Union, Mississippi, US----
                         venue  freq
0         Fast Food Restaurant  0.13
1               Discount Store  0.08
2          Fried Chicken Joint  0.08
3          American Restaurant  0.08
4                  Gas Station  0.08
5                  Pizza Place  0.05
6           Mexican Restaurant  0.05
7                        River  0.03
8                Deli / Bodega  0.03
9                       Bakery  0.03
10              Hardware Store  0.03
11               Big Box Store  0.03
12        Gym / Fitness Center  0.03
13               Grocery Store  0.03
14       Outdoors & Recreation  0.03
15  Construction & Landscaping  0.03
16           Convenience Store  0.03
17            Video Game Store  0.03
18                    Pharmacy  0.03
19                  Shoe Store  0.03
20          Seafood Restaurant  0.03
21                       Hotel  0.03
22               Jewelry Store  0.03
23              Ice Cream Shop  0.03
24

                            venue  freq
0                     Pizza Place  0.33
1                  Discount Store  0.17
2                  Sandwich Place  0.17
3                          Bistro  0.17
4              Mexican Restaurant  0.17
5            Pakistani Restaurant  0.00
6                     Outlet Mall  0.00
7                    Outlet Store  0.00
8               Paella Restaurant  0.00
9                 Paintball Field  0.00
10                            ATM  0.00
11          Outdoors & Recreation  0.00
12  Paper / Office Supplies Store  0.00
13                           Park  0.00
14                        Parking  0.00
15                    Pastry Shop  0.00
16                      Pawn Shop  0.00
17               Pedestrian Plaza  0.00
18                         Palace  0.00
19           Outdoor Supply Store  0.00
20          Performing Arts Venue  0.00
21              Outdoor Sculpture  0.00
22              Other Repair Shop  0.00
23                Other Nightlife  0.00


                   venue  freq
0    American Restaurant  0.08
1         Sandwich Place  0.06
2                  Hotel  0.06
3         Clothing Store  0.05
4     Italian Restaurant  0.05
5   Fast Food Restaurant  0.04
6            Pizza Place  0.04
7                 Lounge  0.03
8            Coffee Shop  0.03
9     Mexican Restaurant  0.03
10                   Bar  0.03
11          Burger Joint  0.03
12        Cosmetics Shop  0.03
13        Hardware Store  0.03
14            Steakhouse  0.03
15           Outlet Mall  0.01
16           Gas Station  0.01
17     Accessories Store  0.01
18                Bakery  0.01
19            Donut Shop  0.01
20                  Café  0.01
21      Greek Restaurant  0.01
22                  Park  0.01
23        Breakfast Spot  0.01
24   Sporting Goods Shop  0.01
25             Pet Store  0.01
26         Bowling Alley  0.01
27              Boutique  0.01
28         Big Box Store  0.01
29                   Gym  0.01


----Washington, Rhode Island, US----


                            venue  freq
0      Construction & Landscaping   0.5
1                        Building   0.5
2                             ATM   0.0
3                            Park   0.0
4                    Outlet Store   0.0
5               Paella Restaurant   0.0
6                 Paintball Field   0.0
7            Pakistani Restaurant   0.0
8                          Palace   0.0
9   Paper / Office Supplies Store   0.0
10                        Parking   0.0
11          Outdoors & Recreation   0.0
12                    Pastry Shop   0.0
13                      Pawn Shop   0.0
14               Pedestrian Plaza   0.0
15                  Pelmeni House   0.0
16          Performing Arts Venue   0.0
17                    Outlet Mall   0.0
18           Outdoor Supply Store   0.0
19             Persian Restaurant   0.0
20              Outdoor Sculpture   0.0
21              Other Repair Shop   0.0
22                Other Nightlife   0.0
23           Other Great Outdoors   0.0


                            venue  freq
0             American Restaurant  0.23
1                  Discount Store  0.15
2                   Grocery Store  0.15
3            Fast Food Restaurant  0.15
4              Photography Studio  0.08
5                  Breakfast Spot  0.08
6                           Trail  0.08
7                            Park  0.08
8                 Paintball Field  0.00
9            Pakistani Restaurant  0.00
10                         Palace  0.00
11                            ATM  0.00
12  Paper / Office Supplies Store  0.00
13                   Outlet Store  0.00
14                        Parking  0.00
15                    Pastry Shop  0.00
16                      Pawn Shop  0.00
17               Pedestrian Plaza  0.00
18              Paella Restaurant  0.00
19          Outdoors & Recreation  0.00
20                    Outlet Mall  0.00
21          Performing Arts Venue  0.00
22           Outdoor Supply Store  0.00
23              Outdoor Sculpture  0.00


**Get top 15 Venues**

Sort the venues in decsending order and get the top 5 venues from the 10 venues found. Create a function to get the top 15 venues around each outbreak hotspot and extract

In [169]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [170]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Province/State']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Province/State'] = grouped['Province/State']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
#neighborhoods_venues_sorted.shape

,Province/State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Abbeville, South Carolina, US",Farm,Stables,Zoo Exhibit,Fair,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Factory,Financial or Legal Service,Fabric Shop,Eye Doctor,Exhibit,Event Space,Event Service
1,"Accomack, Virginia, US",Transportation Service,American Restaurant,Farm,History Museum,Discount Store,Video Store,Business Service,Diner,Convenience Store,Fondue Restaurant,Filipino Restaurant,Event Space,Exhibit,Eye Doctor,Fabric Shop
2,"Ada, Idaho, US",Gun Range,Zoo Exhibit,Financial or Legal Service,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
3,"Adair, Kentucky, US",Fast Food Restaurant,Gas Station,Pizza Place,Hotel,Sandwich Place,Gym,Diner,Café,Mexican Restaurant,Discount Store,Travel & Transport,Grocery Store,Fair,Factory,Fabric Shop
4,"Adair, Missouri, US",Fast Food Restaurant,Pizza Place,American Restaurant,Sandwich Place,Gas Station,Grocery Store,Discount Store,Mexican Restaurant,Coffee Shop,Department Store,State / Provincial Park,Breakfast Spot,Restaurant,Convenience Store,Bowling Alley


**Clustering**

Run k means clustering to sort all the ares (cities) into 7 categories. K-means clustering is a type of unsupervised learning, which is used when you have unlabeled data (i.e., data without defined categories or groups). The goal of this algorithm is to find groups in the data, with the number of groups represented by the variable K. The algorithm works iteratively to assign each data point to one of K groups based on the features that are provided. 

In [171]:
kclusters = 7 # set number of clusters
grouped_clustering = grouped.drop('Province/State', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering) # run k-means clustering
kmeans.labels_[0:7] # check cluster labels generated for each row in the dataframe

array([2, 2, 2, 1, 1, 2, 2], dtype=int32)

In [172]:
# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
merged = df

# merge china_grouped with china_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Province/State'), on='Area')

merged.head() # check the last columns!

,State,Country,Lat,Long,Area,Cases,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",1715,6.0,Convenience Store,Financial or Legal Service,Ethiopian Restaurant,...,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
1,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",5141,4.0,Resort,Golf Course,Lake,...,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Fabric Shop,Eye Doctor,Exhibit,Event Space
2,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",851,2.0,Restaurant,Food,Zoo Exhibit,...,Farmers Market,Farm,Falafel Restaurant,Fair,Eye Doctor,Fabric Shop,Filipino Restaurant,Exhibit,Event Space,Event Service
3,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",638,2.0,Scenic Lookout,Park,Gas Station,...,Farmers Market,Farm,Falafel Restaurant,Fair,Zoo Exhibit,Field,Eye Doctor,Exhibit,Event Space,Event Service
4,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",1580,1.0,Food,Discount Store,Grocery Store,...,American Restaurant,Gas Station,Fair,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Zoo Exhibit,Factory,Fabric Shop


In [173]:
print(merged.shape)

(1879, 22)


In [174]:
merged = merged.dropna() #drop the values from the dataset that were not able to find any venues. This is a limitation of the forsquare api
merged.head()

,State,Country,Lat,Long,Area,Cases,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",1715,6.0,Convenience Store,Financial or Legal Service,Ethiopian Restaurant,...,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
1,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",5141,4.0,Resort,Golf Course,Lake,...,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Factory,Fabric Shop,Eye Doctor,Exhibit,Event Space
2,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",851,2.0,Restaurant,Food,Zoo Exhibit,...,Farmers Market,Farm,Falafel Restaurant,Fair,Eye Doctor,Fabric Shop,Filipino Restaurant,Exhibit,Event Space,Event Service
3,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",638,2.0,Scenic Lookout,Park,Gas Station,...,Farmers Market,Farm,Falafel Restaurant,Fair,Zoo Exhibit,Field,Eye Doctor,Exhibit,Event Space,Event Service
4,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",1580,1.0,Food,Discount Store,Grocery Store,...,American Restaurant,Gas Station,Fair,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Zoo Exhibit,Factory,Fabric Shop


In [175]:
merged['Area'] = merged['Area'].str.replace("'", "")

filter = merged['Area'].str.contains("'")
print(merged.loc[filter,'Area'])

Series([], Name: Area, dtype: object)


In [176]:
merged.shape

(1720, 22)

In [177]:
merged['Cluster Labels'].value_counts()

2.0    901
1.0    615
5.0     57
3.0     50
0.0     49
4.0     27
6.0     21
Name: Cluster Labels, dtype: int64

**Create Folium Map to show Results**

In [178]:
#set Lat/Long for middle of US as default starting point of map 
latitude = 37.926868
longitude = -99.212788 

#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Lat'], merged['Long'], merged['Area'], merged['Cluster Labels']):
    folium.CircleMarker(
        [lat, lon],
        radius=2,
        color=rainbow[int(cluster - 1)],
        fill=True,
        fill_color=rainbow[int(cluster - 1)],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Results and Conlusion: 


<br>**Results:**
<br>The results provided above show that almost all areas have been assigned to cluster '2' and '1' (purple and blue). Because the k-means clustering is placing all the cities into 2 clusters, this may mean that the data contains outliers, the density spread of data points across the data space is different, and/or the data points follow non-convex shapes.
<br>**Possible causes for this**
<br>-Within the radius defined by the forsquare api, the venues might be too similar to each other. 
<br>-The top venues in most locations across the US have really similar venues(resturants, stores, etc). Venues are not the proper feature to find distinctions in different cities. 
<br>-K-means is not the best clustering algorithm to use to put the cities into different categories.

<br> **Conclusion:**
<br>**Question:** What are the top venues in global outbreak areas and do certain venues have any relation with the number of Coronavirus outbreaks? 
<br>**Our Findings:** We can conclude that venues are not the best way to cluster corona-virus outbreak cities in the US as we were not able to group cities in clusters without overfitting. 